In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Reshape, Dropout
from tensorflow.keras.layers import Bidirectional, RepeatVector, Dot, Activation
from tensorflow.keras.layers import Concatenate, Flatten

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

import numpy as np
import pandas as pd
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt
from datetime import datetime
from utils import *


Bad key "text.kerning_factor" on line 4 in
/home/zafir/miniconda3/envs/tensorflow/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [21]:
train_encoder_input_data = np.load('./data/third-order/Centar/train_encoder_input_data.npy')
train_decoder_input_data = np.load('./data/third-order/Centar/train_decoder_input_data.npy')
train_decoder_target_data = np.load('./data/third-order/Centar/train_decoder_target_data.npy')

valid_encoder_input_data = np.load('./data/third-order/Centar/valid_encoder_input_data.npy')
valid_decoder_input_data = np.load('./data/third-order/Centar/valid_decoder_input_data.npy')
valid_decoder_target_data = np.load('./data/third-order/Centar/valid_decoder_target_data.npy')

test_encoder_input_data = np.load('./data/third-order/Centar/test_encoder_input_data.npy')
test_decoder_input_data = np.load('./data/third-order/Centar/test_decoder_input_data.npy')
test_decoder_target_data = np.load('./data/third-order/Centar/test_decoder_target_data.npy')
test_decoder_target_data = test_decoder_target_data.flatten()

In [3]:
Tx, encoder_input_dim = (train_encoder_input_data.shape[1], 
                         train_encoder_input_data.shape[2])
    
Ty, decoder_input_dim = (train_decoder_input_data.shape[1], 
                         train_decoder_input_data.shape[2])

decoder_output_dim = train_decoder_target_data.shape[2]

In [4]:
batch_size = 64
epochs = 150
patience = 10

### Standard

First, using the best hyperparameters found during the random search, we build a model and train it.

In [5]:
K.clear_session()

# ------------------- SHARED LAYERS ---------------------
encoder_lstm = LSTM(64, return_state=True, 
                      name='encoder_lstm')
decoder_lstm = LSTM(64, return_state=True, 
                    return_sequences=True, name='decoder_lstm')
decoder_dense = Dense(decoder_output_dim, 
                      activation='linear', name='decoder_dense')

# Since the best standard model was not stacked and the dense dropout
# rate was 0, we basically can remove the Dropout layers.

# -------------------- TRAIN MODEL ----------------------
encoder_inputs = Input(shape=(Tx, encoder_input_dim), 
                       name='encoder_inputs')

# Obtain the hidden states of the encoder
_, h, c = encoder_lstm(encoder_inputs)

decoder_inputs = Input(shape=(Ty, decoder_input_dim), 
                       name='decoder_inputs')

# Obtain the outputs of the decoder (we don't care about
# the hidden states during training)
x, _, _ = decoder_lstm(decoder_inputs, initial_state=[h, c])
decoder_outputs = decoder_dense(x)

model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)
optimizer = Adam(learning_rate=0.000219)
model.compile(optimizer=optimizer, loss='mse')

Next, we train the model and save the best version

In [6]:
!rm -rf "./logs/standard"

In [7]:
model.fit(x=[train_encoder_input_data, 
            train_decoder_input_data], 
          y=train_decoder_target_data,
          validation_data=([
            valid_encoder_input_data,
            valid_decoder_input_data],
            valid_decoder_target_data),
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[EarlyStopping(monitor='val_loss', 
                                  patience=patience, 
                                  verbose=1),
                     ModelCheckpoint('./checkpoints/standard.h5', 
                                     save_best_only=True),
                     TensorBoard(log_dir="./logs/standard", 
                                 histogram_freq=1)]
         )

Train on 60677 samples, validate on 3388 samples
60677/60677 [==============================] - 68s 1ms/sample - loss: 0.1497 - val_loss: 0.0528


Now, load the best (trained) model, obtain the layers and build an inference model

In [8]:
best_model = load_model('./checkpoints/standard.h5')

In [9]:
encoder_lstm = best_model.get_layer('encoder_lstm')
decoder_lstm = best_model.get_layer('decoder_lstm')
decoder_dense = best_model.get_layer('decoder_dense')

In [27]:
K.clear_session()

# ------------------ INFERENCE MODEL --------------------
encoder_inputs = Input(shape=(Tx, encoder_input_dim), 
                       name='encoder_inputs')

# Obtain the hidden states of the encoder
_, h, c = encoder_lstm(encoder_inputs)

decoder_inputs = Input(shape=(Ty, decoder_input_dim), 
                       name='decoder_inputs')

x = Lambda(lambda z: z[:, 0, :])(decoder_inputs)
x = K.expand_dims(x, axis=1)

outputs = []
for t in range(Ty):
    # Obtain the output and hidden states of the decoder LSTM 
    out, h, c = decoder_lstm(x, initial_state=[h, c])
    out = decoder_dense(out)
    out = Flatten()(out)
    outputs.append(out)

    # Prepare the input for the next timestep by removing the 
    # ground truth value for PM in the previous step and 
    # concatenating the calculated output value. Do this only
    # in case there is a next timestep to be processed.
    if t < Ty - 1:
        x = Lambda(lambda z: z[:, t+1, 1:])(decoder_inputs)
        x = Concatenate(axis=-1)([out, xx])
        x = K.expand_dims(x, axis=1)

model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=outputs)
optimizer = Adam(learning_rate=0.000219)
model.compile(optimizer=optimizer, loss='mse')

In [28]:
def reformat_model_output(y_pred):
    y_pred = np.array(y_pred)
    y_pred = np.swapaxes(y_pred, 0, 1)
    y_pred = y_pred.flatten()
    return y_pred

In [29]:
y_pred = model.predict([test_encoder_input_data, test_decoder_input_data])
y_pred = reformat_model_output(y_pred)

In [30]:
loss = K.eval(tf.keras.losses.mean_squared_error(test_decoder_target_data, y_pred))
loss

0.48393303